In [4]:
from PIL import Image, ImageDraw
import numpy as np
import random
from threading import Thread
import os

import subprocess
import time

In [20]:
a = [int(i/10) for i in image.size]
test_image = image.resize(a, Image.ANTIALIAS)
test_image.save('vou.png', optimize = True, quality = 95)

#foo.save("path\\to\\save\\image_scaled_opt.jpg",optimize=True,quality=95)

In [21]:
image = Image.open("vou.png")
path_save = 'Good/'

draw = ImageDraw.Draw(image) 
width = image.size[0]
height = image.size[1] 
pix = image.load()

step = 1
size = int(max(height, width)* 2**0.5) + 1
last_size = [width, height]

color_delete = [[255] * 4, [150] * 4]

Name_Good = 'Good_desc.dat'
Name_Bad = 'Bad_desc.dat'

Folder_Good = 'Good'
Folder_Bad = 'Bad'

In [23]:
def simple_rotate_value(i, j, phi):
    phi = phi * np.pi / 180
    x = i * np.cos(phi) + j * np.sin(phi)
    y = -i * np.sin(phi) + j * np.cos(phi)
    return int(x), int(y)


def get_noize():
    return tuple([random.randint(0, 255) for i in range(4)])

def create_noise(pix, i, j, color = None):
    
    error = 0
    color = color if color else 100
    flag = full_delete(pix[i,j], [color]*4)
    
    if flag:
        return get_noize(), flag
    else:
        return pix[i, j], flag
    
    
def full_delete(pix, color, error = 0):
    
    f_error = lambda x, color: np.mean([i - j for i,j in zip(x[:-1], color)])    
    return f_error(pix, color) > error

def delete_color(pix, color_list, error):
    # удаление цвета, больше заданного
    
    if color_list:
        if type(color_list) != list:
            color_list = [[color_list] * 4]
        elif type(color_list[0]) != list:
            color_list = [color_list]
    else:
        color_list = [[255] * 4]

    f_error = lambda x, color = [0, 0, 0, 0]: np.mean([abs(i - j) for i,j in zip(x[:-1], color)])    
    
    flag_error = False
    
    for i in color_list:
        flag_error = abs(f_error(pix, i)) > error
        
        if flag_error:
            break
        
    #print(pix, f_error(pix), color, flag_error)
    return flag_error


def rotate_image(image, noise_image, pix, list_point, path_save, phi):
    # поворот картинки на заданный градус
    
    new_image = noise_image.copy()
    draw = ImageDraw.Draw(new_image)
    
    for i in list_point:
        x, y = simple_rotate_value(i[0] - new_image.size[0]/2, i[1] - new_image.size[1]/2, phi)
        draw.point((size/2 + x, size/2 + y), pix[i[0], i[1]])
        
    new_image.save(path_save)
    
def create_noise_image(path_save = None, image = None, size = None):
    # создание шумной картинки заданного размера
    if image:
        new_image = image.copy()
    else:
        new_image = Image.new('RGBA', (size, size))
    
    size = new_image.size[0]
    draw = ImageDraw.Draw(new_image)
    
    for i in range(size):
        for j in range(size):
            draw.point((i, j), get_noize())
    
    if path_save:
        new_image.save(path_save)
        
    return new_image

def change_image(pix, size, last_size, path = None, color_delete = None):
    # зашумление картинки
    print("path:", path)
    
    new_image = Image.new('RGBA', (size, size))
    new_draw = ImageDraw.Draw(new_image)
    
    bias = [(size - i)/2 for i in last_size]
    list_point = []
    
    for i in range(size):
        for j in range(size):
            if abs(i - size/2) < last_size[0]/2 and abs(j - size/2) < last_size[1]/2:
                new_pix, flag = create_noise(pix, i - bias[0], j - bias[1], color = color_delete)
                new_draw.point((i, j), new_pix)
                
                if not flag:
                    list_point.append((i, j))
                
            else:
                new_draw.point((i, j), get_noize())
            
    if path:
        new_image.save(path)

    return new_image, list_point
    
def create_name(size, number = 1, bias = ' '):
    ret = [number, 0, 0, size, size]
    ret_str = bias
    for i in ret:
        ret_str += str(i) + ' '
    return ret_str[:-1]


def change_value(if_value, else_value, check, flag_work = True):
    if flag_work:
        ret = if_value if if_value == check else else_value
    else:
        ret = if_value if if_value != check else else_value
    return ret

def change_str_to_shell(string):
    
    list_symbol = [' ', '(', ')']
    list_replace = [r'\ ', r'\(', r'\)']
    
    for i, j in zip(list_symbol, list_replace):
        string = string.replace(i, j)
    return string

def rename_file_folder(path_folder, base_name, new_format):
# переименовать все файлы в папке 

    black_name = ''
    for counter, i in enumerate(os.listdir(path_folder)):
        if i != black_name:
            
            new_path = change_str_to_shell(os.path.join(path_folder, base_name) + str(counter) + new_format)
            
            subprocess.call('mv ' + change_str_to_shell(os.path.join(path_folder, i)) + ' ' + new_path, shell = True)
            #print('mv ' + os.path.join(path_folder, i).replace(' ', '\ ') + ' ' + new_path.replace(' ', '\ '))
            
        #fl = open(os.path.join(path_folder, i), 'r')
            #txt = fl.read()
            #fl.close()

            #fl = open(new_path, 'w')
            #fl.write(txt)
            #fl.close()
        
def create_description_file(size, number, name_file, counter = 1):
    # создание файла описания 
    
    name_format = '.bmp'
    
    ret_str = ''
    
    const_list = [counter, 0, 0, size, size]
    const_str = ''
    
    for i in const_list:
        const_str += str(i) + ' '
    const_str = const_str[:-1]
    
    for i in range(number):
        ret_str += name_file + str(i) + name_format + '  ' + const_str + '\n'
    
    fl = open(name_file, 'w')
    fl.write(ret_str[:-1])
    fl.close()

In [24]:
Test = True

format_file = '.bmp'

number_thread = 6

phi_list = np.arange(step, 90, step)
counter = 0

flag_thread = False
Time_start = time.time()
# зашумление исходного файла
image_noise, list_image = change_image(pix, size, last_size, 'test' + format_file, color_delete = 30)
Time_1 = time.time()
print(Time_1 - Time_start)
# создание пустого файла
image_full_noise = create_noise_image(image = image_noise, path_save = 'noise' + format_file)
print(time.time() - Time_1)

path: test.bmp
0.8991982936859131
0.4228949546813965


In [26]:
pix_new = image_noise.load()
error_phi = 5
number_image = 400
# создание (error_phi * 2 / step) картинок с поворотом

for counter, i in enumerate(np.linspace(0, error_phi, number_image)):
    path = path_save + str(counter) + format_file
    rotate_image(image_noise, image_full_noise, pix_new, list_image, path, i)
    

In [181]:
create_description_file(image_noise.size[0]/1.5, len(np.arange(-error_phi, error_phi + step, step)), name_file = Name_Good)

In [245]:
rename_file_folder(Folder_Bad, '', '.bmp')

mv Bad/Без\ названия\ (3).jpeg Bad/0.bmp
mv Bad/Без\ названия\ (2).jpeg Bad/1.bmp


In [9]:
np.linspace(0, 1, 400)

array([0.        , 0.00250627, 0.00501253, 0.0075188 , 0.01002506,
       0.01253133, 0.01503759, 0.01754386, 0.02005013, 0.02255639,
       0.02506266, 0.02756892, 0.03007519, 0.03258145, 0.03508772,
       0.03759398, 0.04010025, 0.04260652, 0.04511278, 0.04761905,
       0.05012531, 0.05263158, 0.05513784, 0.05764411, 0.06015038,
       0.06265664, 0.06516291, 0.06766917, 0.07017544, 0.0726817 ,
       0.07518797, 0.07769424, 0.0802005 , 0.08270677, 0.08521303,
       0.0877193 , 0.09022556, 0.09273183, 0.0952381 , 0.09774436,
       0.10025063, 0.10275689, 0.10526316, 0.10776942, 0.11027569,
       0.11278195, 0.11528822, 0.11779449, 0.12030075, 0.12280702,
       0.12531328, 0.12781955, 0.13032581, 0.13283208, 0.13533835,
       0.13784461, 0.14035088, 0.14285714, 0.14536341, 0.14786967,
       0.15037594, 0.15288221, 0.15538847, 0.15789474, 0.160401  ,
       0.16290727, 0.16541353, 0.1679198 , 0.17042607, 0.17293233,
       0.1754386 , 0.17794486, 0.18045113, 0.18295739, 0.18546

In [11]:
15 * 400 / 60

100.0